In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
import scipy.io as sio
import os
import shutil 
from sklearn.cluster import KMeans

In [ ]:
def mkdir(path):
    isExists = os.path.exists(path) 
    if not isExists:
        os.makedirs(path)
        return
    else:
        shutil.rmtree(path)
        os.makedirs(path)
        return

In [ ]:
def combine_data_C(data, C):
    data_with_c = data.copy()
    data_with_c['C'] = C
    return data_with_c

In [ ]:
def nor_data(data2, method):
    if method == 0:
        return (data2 - data2.min())/(data2.max() - data2.min()) ##rescaling
    if method == 1:
        return (data2 - data2.mean())/(data2.max() - data2.min()) ##mean normalization
    if method == 2:
        return (data2-data2.mean())/(data2.std()) ##standarization

In [ ]:
def kmeans(data, clusternumber, name, plotpath, savefile = False, saveplot = True, method = 2):
    

    data2 = pd.DataFrame(data, columns = ['rndinten', 'expinten'])
    #data2 = nor_data(data2, method)
    ss = StandardScaler()
    data2 = ss.fit_transform(data2)

    #print(data2.head()) 

    #sns.set(context="notebook", style="white")
    #sns.lmplot('rndinten', 'expinten', data=data2, fit_reg=False)
    #plt.show()

    #################
    sk_kmeans = KMeans(n_clusters=clusternumber)
    sk_kmeans.fit(data2)

    ##produce cluster label
    sk_C = sk_kmeans.predict(data2)

    #combine label
    data_with_c = combine_data_C(data, sk_C)
    #data_with_c2 = combine_data_C(data2, sk_C)


    ##File Output
    ##outpath = filename.split(".")[0]
    ##if(savefile):
    ##    data_with_c.to_csv(outpath+"_withcluster"+str(clusternumber)+".csv")

    #result plot
    sns.lmplot('rndinten', 'expinten', hue='C', data=data_with_c, fit_reg=False)
    plt.title("indtype = "+name)
    plt.tight_layout()
    if (saveplot):
        plt.savefig(plotpath + name+".png", dpi = 300)
        plt.close()
        return
    else:
        plt.show()
        return

In [ ]:
def csv_group(inputfile, clusternumber, plotpath = "./outplot", method = 2, groupkey = "indtype", outcsvpath = "./outcsv/", save2pdf = True):
    ###
    # 1. Read data as dataframe ###
    #
    data = pd.read_csv(inputfile)

    ###
    # 2.Get various keyvalue ###
    #
    data_to_count = data[groupkey]
    intdic = {}
    for i in data_to_count: #count appe time
        if i in intdic:
            intdic[i] += 1
        else:
            intdic[i] = 1

    intlist = [] #choose desire key
    for i in intdic:
        if intdic[i] >= clusternumber: 
            intlist.append(i)
    intlist.sort() 
    #print(intlist)  ##intlist, len = 523 - 4, dtype = int

    ###
    # 3. Write intlist to csv file
    indtype_df = pd.DataFrame({'avail_ind':intlist})
    indtype_df.to_csv("availible_indtype.csv")

    ###
    # 4. 

    if (!save2pdf):
        plotpath += "/c" + str(clusternumber) + "_m" + str(method) #make a new dir
        mkdir(plotpath)
        plotpath +=  "/ind="
    
    num_int = len(intlist)
    for i in range(num_int):
        intkey = intlist[i]
        data_to_processed = data[data[groupkey] == intkey]
        ##data_to_write.to_csv(outcsvpath + "ind=" + str(intkey) + ".csv") ## "./outcsv/ind=####.csv"
        kmeans(data_to_processed, clusternumber, str(intkey), plotpath)


    return 

In [ ]:
infile = "2001_industry.csv"
num = 2
csv_group(infile, num)

In [ ]:
! git add .
! git commit -m "2.3update1"
! git push